<a href="https://colab.research.google.com/github/husainal1/fpl-player-performance-prediction/blob/main/Predicting_EPL_Player_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# basic setup - install libraries
!pip install pandas==2.2.2 numpy==1.26.4 requests==2.32.3 tqdm==4.66.5 scikit-learn==1.5.2 xgboost==2.1.1

In [ ]:
if 'captain_candidate' in locals():
    print(captain_candidate['web_name'])
else:
    print("Captain candidate not determined yet. Please run the previous cell.")

Captain candidate not determined yet. Please run the previous cell.


In [ ]:
# Find the player with the highest predicted points in the selected squad
if 'selected_squad_df' in locals() and not selected_squad_df.empty:
    captain_candidate = selected_squad_df.loc[selected_squad_df['pred_next_points'].idxmax()]

    print("\nRecommended Captain:")
    # Ensure the columns exist before attempting to display
    display_cols = ['web_name', 'team', 'position', 'pred_next_points']
    existing_cols = [col for col in display_cols if col in captain_candidate.index]

    if len(existing_cols) == len(display_cols):
        display(captain_candidate[existing_cols])
    else:
        print(f"Warning: Not all required display columns found for captain candidate. Available columns: {captain_candidate.index.tolist()}")
        # Attempt to display with available relevant columns
        relevant_cols = ['web_name', 'team', 'position', 'pred_next_points']
        available_relevant_cols = [col for col in relevant_cols if col in captain_candidate.index]
        if available_relevant_cols:
             display(captain_candidate[available_relevant_cols])
        else:
             print("No relevant columns found for captain candidate.")

else:
    print("selected_squad_df DataFrame is not available or is empty. Cannot recommend a captain.")

In [ ]:
#imports - pandas/numpy for data, requests for api, xgboost + sklearn for model
import pandas as pd, numpy as np, requests, time, json, math, datetime as dt
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 200)


In [ ]:
# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

In [ ]:
# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

players.head()


,player_id,first_name,second_name,web_name,team_id,element_type,name,short_name,strength,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,1,David,Raya Martín,Raya,1,1,Arsenal,ARS,4,1350,1350,1290,1300
1,2,Kepa,Arrizabalaga Revuelta,Arrizabalaga,1,1,Arsenal,ARS,4,1350,1350,1290,1300
2,3,Karl,Hein,Hein,1,1,Arsenal,ARS,4,1350,1350,1290,1300
3,4,Tommy,Setford,Setford,1,1,Arsenal,ARS,4,1350,1350,1290,1300
4,5,Gabriel,dos Santos Magalhães,Gabriel,1,2,Arsenal,ARS,4,1350,1350,1290,1300


In [ ]:
# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df


In [ ]:
# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

hist.head()


fetching players: 100%|██████████| 740/740 [00:58<00:00, 12.64it/s]


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,player_id
0,1,9,14,10,0,2025-08-17 15:30:00+00:00,0,1,1,False,90,0,0,1,0,0,0,0,1,0,7,3,38,49.2,0.0,0.0,4.9,1,13,0,0,1,0.00,0.00,0.00,1.52,55,0,1531911,0,0,1
1,1,11,11,6,1,2025-08-23 16:30:00+00:00,5,0,2,False,90,0,0,1,0,0,0,0,0,0,1,0,28,13.4,0.0,0.0,1.3,0,3,0,0,1,0.00,0.00,0.00,0.17,55,218659,2284634,277339,58680,1
2,1,25,12,2,0,2025-08-31 15:30:00+00:00,1,0,3,False,90,0,0,0,1,0,0,0,0,0,2,0,12,20.0,10.0,0.0,3.0,0,12,0,0,1,0.00,0.02,0.02,0.52,55,-12311,2406964,146739,159050,1
3,1,31,16,6,1,2025-09-13 11:30:00+00:00,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752,1
4,2,9,14,0,0,2025-08-17 15:30:00+00:00,0,1,1,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,45,0,90618,0,0,2


In [ ]:
# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']
hist[['web_name','round','total_points','minutes','team_strength_diff']].head(10)


,web_name,round,total_points,minutes,team_strength_diff
0,Raya,1,10,90,1
1,Raya,2,6,90,2
2,Raya,3,2,90,-1
3,Raya,4,6,90,1
4,Arrizabalaga,1,0,0,1
5,Arrizabalaga,2,0,0,2
6,Arrizabalaga,3,0,0,-1
7,Arrizabalaga,4,0,0,1
8,Hein,1,0,0,1
9,Hein,2,0,0,2


In [ ]:
# add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

fe = add_player_features(hist)


/tmp/ipython-input-4071984307.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
/tmp/ipython-input-4071984307.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()
/tmp/ipython-input-4071984307.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

In [ ]:
# looking to predict NEXT match total_points
fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# drop rows that don’t have enough history/future
model_df = fe.dropna(subset=['y_next_points','total_points_lag1','minutes_lag1']).copy()
model_df.shape


/tmp/ipython-input-1726014050.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)


(1395, 160)

In [ ]:
exclude = {'y_next_points','total_points','kickoff_time','web_name','opp_name','opp_short_name',
           'opp_team_id','team_id','opponent_team','name','short_name'}
feature_cols = [c for c in model_df.columns if c not in exclude and c != 'was_home'
                and pd.api.types.is_numeric_dtype(model_df[c])]

X = model_df[feature_cols].fillna(0)
y = model_df['y_next_points'].astype(float)
groups = model_df['player_id']

# baseline = 3 game avg
baseline = model_df['total_points_roll3_mean'].fillna(model_df['total_points_lag1'])


In [ ]:
# groupkfold so same player doesn't leak train/val
gkf = GroupKFold(n_splits=5)
oof_pred = np.zeros(len(model_df))

for tr, va in gkf.split(X, y, groups):
    model = XGBRegressor(
        n_estimators=600, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, n_jobs=-1, tree_method="hist"
    )
    model.fit(X.iloc[tr], y.iloc[tr], eval_set=[(X.iloc[va], y.iloc[va])], verbose=False)
    oof_pred[va] = model.predict(X.iloc[va])

print("Model MAE:", mean_absolute_error(y, oof_pred))
print("Baseline MAE:", mean_absolute_error(y, baseline))


Model MAE: 1.0945412781540602
Baseline MAE: 1.3512544802867383


In [ ]:
final_model = XGBRegressor(
    n_estimators=800, learning_rate=0.04, max_depth=6,
    subsample=0.9, colsample_bytree=0.9,
    random_state=42, n_jobs=-1, tree_method="hist"
)
final_model.fit(X, y, verbose=False)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.04, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=800, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Next gameweek fixtures
upcoming = fixtures.copy()
next_gw = upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].min()

if pd.isna(next_gw):
    print("No upcoming gameweek found yet.")
else:
    upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

    # Add print statements to check upcoming_next
    print("\nUpcoming next gameweek fixtures (upcoming_next):")
    print("Shape:", upcoming_next.shape)
    print("Columns:", upcoming_next.columns)
    display(upcoming_next.head())


    # map each team to (opp_team_id, was_home)
    home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
    home['was_home'] = 1
    away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
    away['was_home'] = 0
    team_next = pd.concat([home, away], ignore_index=True)

    # Add print statements to check team_next
    print("\nTeam next fixture mapping (team_next):")
    print("Shape:", team_next.shape)
    print("Columns:", team_next.columns)
    display(team_next.head())


    # join fixture mapping to latest
    # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
    latest = latest.merge(team_next, on='team_id', how='left')

    # Add print statement to check latest columns after merging with team_next
    print("\nColumns of latest after merge with team_next:")
    print(latest.columns)
    print("\nHead of latest after merge with team_next:")
    display(latest.head())

    # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
    if 'opp_team_id_y' not in latest.columns or latest['opp_team_id_y'].isnull().all():
        print("Warning: 'opp_team_id_y' column is missing or all null after merging with team_next. Cannot merge with opponent strengths.")
    else:
        # bring opponent strengths
        opp_strengths = teams.rename(columns={
            'team_id':'opp_team_id',
            'strength':'opp_strength',
            'strength_defence_home':'opp_strength_defence_home',
            'strength_defence_away':'opp_strength_defence_away'
        })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

        # Now merge latest with opp_strengths on 'opp_team_id_y'
        latest = latest.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

        # Add print statement to check latest columns after merging with opp_strengths
        print("\nColumns of latest after merge with opp_strengths:")
        print(latest.columns)
        print("\nHead of latest after merge with opp_strengths:")
        display(latest.head())


        # recompute venue-aware attack vs defence diff for the UPCOMING match
        # Use the correct suffixed column names for the calculation
        latest['attack_v_def_diff'] = np.where(
            latest['was_home_y'] == 1,
            latest['strength_attack_home_x'] - latest['opp_strength_defence_away_y'],
            latest['strength_attack_away_x'] - latest['opp_strength_defence_home_y']
        )

        # Predict
        # use same feature set you trained with
        # Ensure X_pred has the same columns as X used for training
        X_pred = latest.reindex(columns=feature_cols).fillna(0)
        latest['pred_next_points'] = final_model.predict(X_pred)

        # tidy columns for viewing
        TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
        POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
        # Use the correct suffixed column names for mapping
        latest['team'] = latest['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
        latest['opp']  = latest['opp_team_id_y'].map(TEAM_MAP)
        latest['position'] = latest['element_type_x'].map(POS_MAP) # element_type got suffix _x


        latest['pred_next_points'] = latest['pred_next_points'].round(2)

        # final table (pandas)
        # Use the correct suffixed column names for the final table
        final_tbl = latest[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                        .sort_values('pred_next_points', ascending=False) \
                        .reset_index(drop=True)

        # show top 50
        print(final_tbl.head(50))

Columns of latest before first merge (with players):
Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'modified',
       ...
       'expected_goals_roll8_sum', 'expected_assists_roll8_mean',
       'expected_assists_roll8_sum', 'expected_goal_involvements_roll8_mean',
       'expected_goal_involvements_roll8_sum', 'played_last_match',
       'played_last3_pct', 'attack_v_def_diff', 'month', 'dow'],
      dtype='object', length=159)

Head of latest before first merge (with players):


/tmp/ipython-input-4071984307.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
/tmp/ipython-input-4071984307.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()
/tmp/ipython-input-4071984307.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,player_id,team_id,web_name,element_type,strength,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,opp_team_id,opp_strength,opp_strength_defence_home,opp_strength_defence_away,team_strength_diff,total_points_lag1,total_points_lag2,total_points_lag3,minutes_lag1,minutes_lag2,minutes_lag3,goals_scored_lag1,goals_scored_lag2,goals_scored_lag3,assists_lag1,assists_lag2,assists_lag3,ict_index_lag1,ict_index_lag2,ict_index_lag3,creativity_lag1,creativity_lag2,creativity_lag3,influence_lag1,influence_lag2,influence_lag3,threat_lag1,threat_lag2,threat_lag3,expected_goals_lag1,expected_goals_lag2,expected_goals_lag3,expected_assists_lag1,expected_assists_lag2,expected_assists_lag3,expected_goal_involvements_lag1,expected_goal_involvements_lag2,expected_goal_involvements_lag3,total_points_roll3_mean,total_points_roll3_sum,minutes_roll3_mean,minutes_roll3_sum,goals_scored_roll3_mean,goals_scored_roll3_sum,assists_roll3_mean,assists_roll3_sum,ict_index_roll3_mean,ict_index_roll3_sum,creativity_roll3_mean,creativity_roll3_sum,influence_roll3_mean,influence_roll3_sum,threat_roll3_mean,threat_roll3_sum,expected_goals_roll3_mean,expected_goals_roll3_sum,expected_assists_roll3_mean,expected_assists_roll3_sum,expected_goal_involvements_roll3_mean,expected_goal_involvements_roll3_sum,total_points_roll5_mean,total_points_roll5_sum,minutes_roll5_mean,minutes_roll5_sum,goals_scored_roll5_mean,goals_scored_roll5_sum,assists_roll5_mean,assists_roll5_sum,ict_index_roll5_mean,ict_index_roll5_sum,creativity_roll5_mean,creativity_roll5_sum,influence_roll5_mean,influence_roll5_sum,threat_roll5_mean,threat_roll5_sum,expected_goals_roll5_mean,expected_goals_roll5_sum,expected_assists_roll5_mean,expected_assists_roll5_sum,expected_goal_involvements_roll5_mean,expected_goal_involvements_roll5_sum,total_points_roll8_mean,total_points_roll8_sum,minutes_roll8_mean,minutes_roll8_sum,goals_scored_roll8_mean,goals_scored_roll8_sum,assists_roll8_mean,assists_roll8_sum,ict_index_roll8_mean,ict_index_roll8_sum,creativity_roll8_mean,creativity_roll8_sum,influence_roll8_mean,influence_roll8_sum,threat_roll8_mean,threat_roll8_sum,expected_goals_roll8_mean,expected_goals_roll8_sum,expected_assists_roll8_mean,expected_assists_roll8_sum,expected_goal_involvements_roll8_mean,expected_goal_involvements_roll8_sum,played_last_match,played_last3_pct,attack_v_def_diff,month,dow
3,1,31,16,6,1,2025-09-13 11:30:00+00:00,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752,1,1,Raya,1,4,1350,1350,1290,1300,16,3,1180,1180,1,2.0,6.0,10.0,90.0,90.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.3,4.9,10.0,0.0,0.0,20.0,13.4,49.2,0.0,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,6.000000,18.0,90.0,270.0,0.0,0.0,0.0,0.0,3.066667,9.2,3.333333,10.0,27.533333,82.6,0.0,0.0,0.00,0.00,0.006667,0.02,0.006667,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,170,9,5
7,2,31,16,0,1,2025-09-13 11:30:00+00:00,3,0,4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,44,-6913,76258,1271,8184,2,1,Arrizabalaga,1,4,1350,1350,1290,1300,16,3,1180,1180,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0


Columns of latest after first merge (with players):
Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'modified',
       ...
       'month', 'dow', 'team_id_y', 'web_name_y', 'element_type_y',
       'strength_y', 'strength_attack_home_y', 'strength_attack_away_y',
       'strength_defence_home_y', 'strength_defence_away_y'],
      dtype='object', length=167)

Head of latest after first merge (with players):


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,player_id,team_id_x,web_name_x,element_type_x,strength_x,strength_attack_home_x,strength_attack_away_x,strength_defence_home_x,strength_defence_away_x,opp_team_id,opp_strength,opp_strength_defence_home,opp_strength_defence_away,team_strength_diff,total_points_lag1,total_points_lag2,total_points_lag3,minutes_lag1,minutes_lag2,minutes_lag3,goals_scored_lag1,goals_scored_lag2,goals_scored_lag3,assists_lag1,assists_lag2,assists_lag3,ict_index_lag1,ict_index_lag2,ict_index_lag3,creativity_lag1,creativity_lag2,creativity_lag3,influence_lag1,influence_lag2,influence_lag3,threat_lag1,threat_lag2,threat_lag3,expected_goals_lag1,expected_goals_lag2,expected_goals_lag3,expected_assists_lag1,expected_assists_lag2,expected_assists_lag3,expected_goal_involvements_lag1,expected_goal_involvements_lag2,expected_goal_involvements_lag3,total_points_roll3_mean,total_points_roll3_sum,minutes_roll3_mean,minutes_roll3_sum,goals_scored_roll3_mean,goals_scored_roll3_sum,assists_roll3_mean,assists_roll3_sum,ict_index_roll3_mean,ict_index_roll3_sum,creativity_roll3_mean,creativity_roll3_sum,influence_roll3_mean,influence_roll3_sum,threat_roll3_mean,threat_roll3_sum,expected_goals_roll3_mean,expected_goals_roll3_sum,expected_assists_roll3_mean,expected_assists_roll3_sum,expected_goal_involvements_roll3_mean,expected_goal_involvements_roll3_sum,total_points_roll5_mean,total_points_roll5_sum,minutes_roll5_mean,minutes_roll5_sum,goals_scored_roll5_mean,goals_scored_roll5_sum,assists_roll5_mean,assists_roll5_sum,ict_index_roll5_mean,ict_index_roll5_sum,creativity_roll5_mean,creativity_roll5_sum,influence_roll5_mean,influence_roll5_sum,threat_roll5_mean,threat_roll5_sum,expected_goals_roll5_mean,expected_goals_roll5_sum,expected_assists_roll5_mean,expected_assists_roll5_sum,expected_goal_involvements_roll5_mean,expected_goal_involvements_roll5_sum,total_points_roll8_mean,total_points_roll8_sum,minutes_roll8_mean,minutes_roll8_sum,goals_scored_roll8_mean,goals_scored_roll8_sum,assists_roll8_mean,assists_roll8_sum,ict_index_roll8_mean,ict_index_roll8_sum,creativity_roll8_mean,creativity_roll8_sum,influence_roll8_mean,influence_roll8_sum,threat_roll8_mean,threat_roll8_sum,expected_goals_roll8_mean,expected_goals_roll8_sum,expected_assists_roll8_mean,expected_assists_roll8_sum,expected_goal_involvements_roll8_mean,expected_goal_involvements_roll8_sum,played_last_match,played_last3_pct,attack_v_def_diff,month,dow,team_id_y,web_name_y,element_type_y,strength_y,strength_attack_home_y,strength_attack_away_y,strength_defence_home_y,strength_defence_away_y
0,1,31,16,6,1,2025-09-13 11:30:00+00:00,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752,1,1,Raya,1,4,1350,1350,1290,1300,16,3,1180,1180,1,2.0,6.0,10.0,90.0,90.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.3,4.9,10.0,0.0,0.0,20.0,13.4,49.2,0.0,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,6.000000,18.0,90.0,270.0,0.0,0.0,0.0,0.0,3.066667,9.2,3.333333,10.0,27.533333,82.6,0.0,0.0,0.00,0.00,0.006667,0.02,0.006667,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,170,9,5,1,Raya,1,4,1350,1350,1290,1300
1,2,31,16,0,1,2025-09-13 11:30:00+00:00,3,0,4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,44,-6913,76258,1271,8184,2,1,Arrizabalaga,1,4,1350,1350,1290,1300,16,3,1180,1180,1,0.0,0.0,0.0,0.0,0

...


Columns of latest after merge with team_next:
Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home_x',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'modified',
       ...
       'team_id_y', 'web_name_y', 'element_type_y', 'strength_y',
       'strength_attack_home_y', 'strength_attack_away_y',
       'strength_defence_home_y', 'strength_defence_away_y', 'opp_team_id_y',
       'was_home_y'],
      dtype='object', length=169)

Head of latest after merge with team_next:


,element,fixture,opponent_team,total_points,was_home_x,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,player_id,team_id,web_name_x,element_type_x,strength_x,strength_attack_home_x,strength_attack_away_x,strength_defence_home_x,strength_defence_away_x,opp_team_id_x,opp_strength,opp_strength_defence_home,opp_strength_defence_away,team_strength_diff,total_points_lag1,total_points_lag2,total_points_lag3,minutes_lag1,minutes_lag2,minutes_lag3,goals_scored_lag1,goals_scored_lag2,goals_scored_lag3,assists_lag1,assists_lag2,assists_lag3,ict_index_lag1,ict_index_lag2,ict_index_lag3,creativity_lag1,creativity_lag2,creativity_lag3,influence_lag1,influence_lag2,influence_lag3,threat_lag1,threat_lag2,threat_lag3,expected_goals_lag1,expected_goals_lag2,expected_goals_lag3,expected_assists_lag1,expected_assists_lag2,expected_assists_lag3,expected_goal_involvements_lag1,expected_goal_involvements_lag2,expected_goal_involvements_lag3,total_points_roll3_mean,total_points_roll3_sum,minutes_roll3_mean,minutes_roll3_sum,goals_scored_roll3_mean,goals_scored_roll3_sum,assists_roll3_mean,assists_roll3_sum,ict_index_roll3_mean,ict_index_roll3_sum,creativity_roll3_mean,creativity_roll3_sum,influence_roll3_mean,influence_roll3_sum,threat_roll3_mean,threat_roll3_sum,expected_goals_roll3_mean,expected_goals_roll3_sum,expected_assists_roll3_mean,expected_assists_roll3_sum,expected_goal_involvements_roll3_mean,expected_goal_involvements_roll3_sum,total_points_roll5_mean,total_points_roll5_sum,minutes_roll5_mean,minutes_roll5_sum,goals_scored_roll5_mean,goals_scored_roll5_sum,assists_roll5_mean,assists_roll5_sum,ict_index_roll5_mean,ict_index_roll5_sum,creativity_roll5_mean,creativity_roll5_sum,influence_roll5_mean,influence_roll5_sum,threat_roll5_mean,threat_roll5_sum,expected_goals_roll5_mean,expected_goals_roll5_sum,expected_assists_roll5_mean,expected_assists_roll5_sum,expected_goal_involvements_roll5_mean,expected_goal_involvements_roll5_sum,total_points_roll8_mean,total_points_roll8_sum,minutes_roll8_mean,minutes_roll8_sum,goals_scored_roll8_mean,goals_scored_roll8_sum,assists_roll8_mean,assists_roll8_sum,ict_index_roll8_mean,ict_index_roll8_sum,creativity_roll8_mean,creativity_roll8_sum,influence_roll8_mean,influence_roll8_sum,threat_roll8_mean,threat_roll8_sum,expected_goals_roll8_mean,expected_goals_roll8_sum,expected_assists_roll8_mean,expected_assists_roll8_sum,expected_goal_involvements_roll8_mean,expected_goal_involvements_roll8_sum,played_last_match,played_last3_pct,attack_v_def_diff,month,dow,team_id_y,web_name_y,element_type_y,strength_y,strength_attack_home_y,strength_attack_away_y,strength_defence_home_y,strength_defence_away_y,opp_team_id_y,was_home_y
0,1,31,16,6,1,2025-09-13 11:30:00+00:00,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752,1,1,Raya,1,4,1350,1350,1290,1300,16,3,1180,1180,1,2.0,6.0,10.0,90.0,90.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.3,4.9,10.0,0.0,0.0,20.0,13.4,49.2,0.0,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,6.000000,18.0,90.0,270.0,0.0,0.0,0.0,0.0,3.066667,9.2,3.333333,10.0,27.533333,82.6,0.0,0.0,0.00,0.00,0.006667,0.02,0.006667,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,170,9,5,1,Raya,1,4,1350,1350,1290,1300,13,1
1,2,31,16,0,1,2025-09-13 11:30:00+00:00,3,0,4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,44,-6913,76258,1271,8184,2,1,Arrizabalaga,1,4,1350,1350,1290,1300,16


Columns of latest after merge with opp_strengths:
Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home_x',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'modified',
       ...
       'strength_attack_home_y', 'strength_attack_away_y',
       'strength_defence_home_y', 'strength_defence_away_y', 'opp_team_id_y',
       'was_home_y', 'opp_team_id', 'opp_strength_y',
       'opp_strength_defence_home_y', 'opp_strength_defence_away_y'],
      dtype='object', length=173)

Head of latest after merge with opp_strengths:


,element,fixture,opponent_team,total_points,was_home_x,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,player_id,team_id,web_name_x,element_type_x,strength_x,strength_attack_home_x,strength_attack_away_x,strength_defence_home_x,strength_defence_away_x,opp_team_id_x,opp_strength_x,opp_strength_defence_home_x,opp_strength_defence_away_x,team_strength_diff,total_points_lag1,total_points_lag2,total_points_lag3,minutes_lag1,minutes_lag2,minutes_lag3,goals_scored_lag1,goals_scored_lag2,goals_scored_lag3,assists_lag1,assists_lag2,assists_lag3,ict_index_lag1,ict_index_lag2,ict_index_lag3,creativity_lag1,creativity_lag2,creativity_lag3,influence_lag1,influence_lag2,influence_lag3,threat_lag1,threat_lag2,threat_lag3,expected_goals_lag1,expected_goals_lag2,expected_goals_lag3,expected_assists_lag1,expected_assists_lag2,expected_assists_lag3,expected_goal_involvements_lag1,expected_goal_involvements_lag2,expected_goal_involvements_lag3,total_points_roll3_mean,total_points_roll3_sum,minutes_roll3_mean,minutes_roll3_sum,goals_scored_roll3_mean,goals_scored_roll3_sum,assists_roll3_mean,assists_roll3_sum,ict_index_roll3_mean,ict_index_roll3_sum,creativity_roll3_mean,creativity_roll3_sum,influence_roll3_mean,influence_roll3_sum,threat_roll3_mean,threat_roll3_sum,expected_goals_roll3_mean,expected_goals_roll3_sum,expected_assists_roll3_mean,expected_assists_roll3_sum,expected_goal_involvements_roll3_mean,expected_goal_involvements_roll3_sum,total_points_roll5_mean,total_points_roll5_sum,minutes_roll5_mean,minutes_roll5_sum,goals_scored_roll5_mean,goals_scored_roll5_sum,assists_roll5_mean,assists_roll5_sum,ict_index_roll5_mean,ict_index_roll5_sum,creativity_roll5_mean,creativity_roll5_sum,influence_roll5_mean,influence_roll5_sum,threat_roll5_mean,threat_roll5_sum,expected_goals_roll5_mean,expected_goals_roll5_sum,expected_assists_roll5_mean,expected_assists_roll5_sum,expected_goal_involvements_roll5_mean,expected_goal_involvements_roll5_sum,total_points_roll8_mean,total_points_roll8_sum,minutes_roll8_mean,minutes_roll8_sum,goals_scored_roll8_mean,goals_scored_roll8_sum,assists_roll8_mean,assists_roll8_sum,ict_index_roll8_mean,ict_index_roll8_sum,creativity_roll8_mean,creativity_roll8_sum,influence_roll8_mean,influence_roll8_sum,threat_roll8_mean,threat_roll8_sum,expected_goals_roll8_mean,expected_goals_roll8_sum,expected_assists_roll8_mean,expected_assists_roll8_sum,expected_goal_involvements_roll8_mean,expected_goal_involvements_roll8_sum,played_last_match,played_last3_pct,attack_v_def_diff,month,dow,team_id_y,web_name_y,element_type_y,strength_y,strength_attack_home_y,strength_attack_away_y,strength_defence_home_y,strength_defence_away_y,opp_team_id_y,was_home_y,opp_team_id,opp_strength_y,opp_strength_defence_home_y,opp_strength_defence_away_y
0,1,31,16,6,1,2025-09-13 11:30:00+00:00,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752,1,1,Raya,1,4,1350,1350,1290,1300,16,3,1180,1180,1,2.0,6.0,10.0,90.0,90.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.3,4.9,10.0,0.0,0.0,20.0,13.4,49.2,0.0,0.0,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,6.000000,18.0,90.0,270.0,0.0,0.0,0.0,0.0,3.066667,9.2,3.333333,10.0,27.533333,82.6,0.0,0.0,0.00,0.00,0.006667,0.02,0.006667,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,170,9,5,1,Raya,1,4,1350,1350,1290,1300,13,1,13,4,1300,1380
1,2,31,16,0,1,2025-09-13 11:30:00+00:00,3,0,4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.

       web_name_x position team  opp  was_home_y  pred_next_points
0          Senesi      DEF  BOU  NEW           1              8.50
1        Andersen      DEF  FUL  BRE           1              7.67
2         De Ligt      DEF  MUN  CHE           1              7.43
3           Rodon      DEF  LEE  WOL           0              6.74
4         Gabriel      DEF  ARS  MCI           1              6.71
5        Bruno G.      MID  NEW  BOU           0              6.62
6        Dúbravka       GK  BUR  NFO           1              6.39
7        Petrović       GK  BOU  NEW           1              6.34
8          Ndiaye      MID  EVE  LIV           0              6.31
9       Tavernier      MID  BOU  NEW           1              6.20
10     Donnarumma       GK  MCI  ARS           0              5.78
11        M.Salah      MID  LIV  EVE           1              5.78
12       Chalobah      DEF  CHE  MUN           0              5.77
13       Truffert      DEF  BOU  NEW           1              

In [ ]:
# Define Big 6
BIG6 = ["ARS", "CHE", "LIV", "MCI", "MUN", "TOT"]

# Exclude Big 6 players
underdogs = final_tbl[~final_tbl['team'].isin(BIG6)].copy()

# Sort by predicted points
underdogs = underdogs.sort_values('pred_next_points', ascending=False)

# Top 5 bargains
top5_underdogs = underdogs.head(5)

print("Top 5 bargain underdogs (non-Big 6 teams):")
# Use the correct column name 'web_name_x'
print(top5_underdogs[['web_name_x','team','position','pred_next_points']])


print("\n----------------------------------------\n")

# Find best forwards for their price
# Ensure 'latest' DataFrame is accessible and contains 'value' (price), 'position', and 'web_name_x'
if 'value' in latest.columns and 'position' in latest.columns and 'web_name_x' in latest.columns:
    forwards = latest[latest['position'] == 'FWD'].copy()

    # Calculate points per price, handling potential division by zero
    forwards['points_per_price'] = np.where(forwards['value'] > 0, forwards['pred_next_points'] / (forwards['value'] / 10.0), 0) # Value is in 0.1M increments

    # Sort by points per price
    best_value_forwards = forwards.sort_values('points_per_price', ascending=False)

    print("Top forwards for their price:")
    # Display relevant columns using correct names
    print(best_value_forwards[['web_name_x','team','position','value','pred_next_points','points_per_price']].head(20))
else:
    print("Required columns ('value', 'position', or 'web_name_x') not found in the latest data.")

print("\n----------------------------------------\n")

# Find best defenders for their price
# Ensure 'latest' DataFrame is accessible and contains 'value' (price), 'position', and 'web_name_x'
if 'value' in latest.columns and 'position' in latest.columns and 'web_name_x' in latest.columns:
    defenders = latest[latest['position'] == 'DEF'].copy()

    # Calculate points per price, handling potential division by zero
    defenders['points_per_price'] = np.where(defenders['value'] > 0, defenders['pred_next_points'] / (defenders['value'] / 10.0), 0) # Value is in 0.1M increments

    # Sort by points per price
    best_value_defenders = defenders.sort_values('points_per_price', ascending=False)

    print("Top defenders for their price:")
    # Display relevant columns using correct names
    print(best_value_defenders[['web_name_x','team','position','value','pred_next_points','points_per_price']].head(20))
else:
    print("Required columns ('value', 'position', or 'web_name_x') not found in the latest data.")

print("\n----------------------------------------\n")

# Find best midfielders for their price
# Ensure 'latest' DataFrame is accessible and contains 'value' (price), 'position', and 'web_name_x'
if 'value' in latest.columns and 'position' in latest.columns and 'web_name_x' in latest.columns:
    midfielders = latest[latest['position'] == 'MID'].copy()

    # Calculate points per price, handling potential division by zero
    midfielders['points_per_price'] = np.where(midfielders['value'] > 0, midfielders['pred_next_points'] / (midfielders['value'] / 10.0), 0) # Value is in 0.1M increments

    # Sort by points per price
    best_value_midfielders = midfielders.sort_values('points_per_price', ascending=False)

    print("Top midfielders for their price:")
    # Display relevant columns using correct names
    print(best_value_midfielders[['web_name_x','team','position','value','pred_next_points','points_per_price']].head(20))
else:
    print("Required columns ('value', 'position', or 'web_name_x') not found in the latest data.")

Top 5 bargain underdogs (non-Big 6 teams):
  web_name_x team position  pred_next_points
0     Senesi  BOU      DEF              8.50
1   Andersen  FUL      DEF              7.67
3      Rodon  LEE      DEF              6.74
5   Bruno G.  NEW      MID              6.62
6   Dúbravka  BUR       GK              6.39

----------------------------------------

Top forwards for their price:
        web_name_x team position  value  pred_next_points  points_per_price
216         Foster  BUR      FWD     50              4.82          0.964000
96       Evanilson  BOU      FWD     70              5.38          0.768571
559         Isidor  SUN      FWD     55              3.49          0.634545
310           Beto  EVE      FWD     54              3.18          0.588889
695     Kalimuendo  NFO      FWD     60              3.48          0.580000
690  Calvert-Lewin  LEE      FWD     55              3.15          0.572727
282         Mateta  CRY      FWD     75              4.28          0.570667
653  S

In [ ]:
# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))


NameError: name 'add_player_features' is not defined

In [ ]:
# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'hist' is not defined

In [ ]:
# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'players' is not defined

In [ ]:
from tqdm import tqdm

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'players' is not defined

In [ ]:
# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

from tqdm import tqdm

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'get_json' is not defined

In [ ]:
# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

from tqdm import tqdm

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'requests' is not defined

In [ ]:
import requests

# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

from tqdm import tqdm

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd

# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

from tqdm import tqdm

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df

# Build latest snapshot per player
fe = add_player_features(hist)

# next-match target for training
# fe['y_next_points'] = fe.groupby('player_id')['total_points'].shift(-1)

# latest row per player by time
latest = (
    fe.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# Add print statement to inspect latest columns before first merge (with players)
print("Columns of latest before first merge (with players):")
print(latest.columns)
print("\nHead of latest before first merge (with players):")
display(latest.head())


# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away']].copy()

latest = latest.merge(players_min, on='player_id', how='left')

# Add print statement to inspect latest columns after first merge (with players):")
print("\nColumns of latest after first merge (with players):")
print(latest.columns)
print("\nHead of latest after first merge (with players):")
display(latest.head())


# sanity: ensure 'team_id' exists (handle accidental suffixes)
if 'team_id' not in latest.columns:
    for cand in ['team_id_x','team_id_y']:
        if cand in latest.columns:
            latest = latest.rename(columns={cand: 'team_id'})
            break

# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # Add print statements to check upcoming_next
        print(f"\nUpcoming fixtures for Gameweek {next_gw} (upcoming_next):")
        print("Shape:", upcoming_next.shape)
        print("Columns:", upcoming_next.columns)
        display(upcoming_next.head())


        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # Add print statements to check team_next
        print(f"\nTeam next fixture mapping for Gameweek {next_gw} (team_next):")
        print("Shape:", team_next.shape)
        print("Columns:", team_next.columns)
        display(team_next.head())


        # join fixture mapping to latest
        # This merge adds 'opp_team_id_y' and 'was_home_y' to latest
        # Need to create a temporary DataFrame for the merge to avoid modifying 'latest' in the loop
        latest_gw = latest.merge(team_next, on='team_id', how='left')


        # Add print statement to check latest_gw columns after merging with team_next
        print(f"\nColumns of latest_gw after merge with team_next for Gameweek {next_gw}:")
        print(latest_gw.columns)
        print(f"\nHead of latest_gw after merge with team_next for Gameweek {next_gw}:")
        display(latest_gw.head())

        # Check if opp_team_id_y is present and has non-null values before merging with opp_strengths
        if 'opp_team_id_y' not in latest_gw.columns or latest_gw['opp_team_id_y'].isnull().all():
            print(f"Warning: 'opp_team_id_y' column is missing or all null after merging with team_next for Gameweek {next_gw}. Cannot merge with opponent strengths.")
        else:
            # bring opponent strengths
            opp_strengths = teams.rename(columns={
                'team_id':'opp_team_id',
                'strength':'opp_strength',
                'strength_defence_home':'opp_strength_defence_home',
                'strength_defence_away':'opp_strength_defence_away'
            })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

            # Now merge latest_gw with opp_strengths on 'opp_team_id_y'
            latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_y', right_on='opp_team_id', how='left')

            # Add print statement to check latest_gw columns after merging with opp_strengths
            print(f"\nColumns of latest_gw after merge with opp_strengths for Gameweek {next_gw}:")
            print(latest_gw.columns)
            print(f"\nHead of latest_gw after merge with team_strengths for Gameweek {next_gw}:")
            display(latest_gw.head())


            # recompute venue-aware attack vs defence diff for the UPCOMING match
            # Use the correct suffixed column names for the calculation
            latest_gw['attack_v_def_diff'] = np.where(
                latest_gw['was_home_y'] == 1,
                latest_gw['strength_attack_home_x'] - latest_gw['opp_strength_defence_away_y'],
                latest_gw['strength_attack_away_x'] - latest_gw['opp_strength_defence_home_y']
            )

            # Predict
            # use same feature set you trained with
            # Ensure X_pred has the same columns as X used for training
            X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
            latest_gw['pred_next_points'] = final_model.predict(X_pred)

            # tidy columns for viewing
            TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
            POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
            # Use the correct suffixed column names for mapping
            latest_gw['team'] = latest_gw['team_id'].map(TEAM_MAP) # team_id did not get suffix after first merge, but will after second, need team_id_x
            latest_gw['opp']  = latest_gw['opp_team_id_y'].map(TEAM_MAP)
            latest_gw['position'] = latest_gw['element_type_x'].map(POS_MAP) # element_type got suffix _x


            latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

            # final table (pandas)
            # Use the correct suffixed column names for the final table
            final_tbl_gw = latest_gw[['web_name_x','position','team','opp','was_home_y','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

            # show top 50
            print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
            print(final_tbl_gw.head(50))

fetching players: 100%|██████████| 746/746 [00:45<00:00, 16.38it/s]
/tmp/ipython-input-2226186867.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
/tmp/ipython-input-2226186867.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()
/tmp/ipython-input-2226186867.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor per

NameError: name 'np' is not defined

In [ ]:
# ===============================================
#   FPL PREDICTION PIPELINE WITH PRICE BY GW
# ===============================================

import pandas as pd, numpy as np, requests, time, json, math, datetime as dt
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 200)

# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info
players = players_meta[['id','first_name','second_name','web_name','team','element_type', 'value']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time']
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except:
        pass  # if one player fails, skip

hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df


# ---- STEP 1. SETUP PLAYER PRICE HISTORY ----
# hist:  contains player_id, round, value (historical prices)
# players_meta: contains id, now_cost (current price)
# upcoming: fixture table that includes 'event' (gameweek)

# Build price table from history (tenths of £m -> £m)
price_hist = (
    hist.loc[hist['value'].notna(), ['player_id', 'round', 'value']]
        .rename(columns={'round': 'gameweek', 'value': 'price_tenths'})
        .copy()
)
price_hist['price'] = price_hist['price_tenths'] / 10.0
price_hist = price_hist[['player_id', 'gameweek', 'price']]

# Current prices snapshot
current_prices = (
    players_meta[['id', 'now_cost']]
        .rename(columns={'id': 'player_id', 'now_cost': 'price_tenths'})
        .copy()
)
current_prices['price'] = current_prices['price_tenths'] / 10.0
current_prices = current_prices[['player_id', 'price']]

# Determine upcoming gameweeks (future events)
upcoming_gws = sorted(
    fixtures.loc[(~fixtures['finished']) & (fixtures['event'].notna()), 'event'].unique()
)

# Helper to replicate current prices across future GWs
def make_future_price_rows(upcoming_gws, current_prices):
    rows = []
    for gw in upcoming_gws:
        tmp = current_prices.copy()
        tmp['gameweek'] = gw
        rows.append(tmp)
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['player_id','price','gameweek'])

future_prices = make_future_price_rows(upcoming_gws, current_prices)

# Combine past and future prices
price_gw_long = (
    pd.concat([price_hist, future_prices], ignore_index=True)
      .sort_values(['player_id', 'gameweek'])
      .drop_duplicates(['player_id', 'gameweek'], keep='first')
      .reset_index(drop=True)
)

# Optional sanity check
assert not price_gw_long.duplicated(['player_id', 'gameweek']).any()


# Prepare data for model training
fe_train = add_player_features(hist.copy())
fe_train['y_next_points'] = fe_train.groupby('player_id')['total_points'].shift(-1)
model_df = fe_train.dropna(subset=['y_next_points','total_points_lag1','minutes_lag1']).copy()

exclude = {'y_next_points','total_points','kickoff_time','web_name','opp_name','opp_short_name',
           'opp_team_id','team_id','opponent_team','name','short_name'}
feature_cols = [c for c in model_df.columns if c not in exclude and c != 'was_home'
                and pd.api.types.is_numeric_dtype(model_df[c])]

X = model_df[feature_cols].fillna(0)
y = model_df['y_next_points'].astype(float)
groups = model_df['player_id']

# Train final model on all data
final_model = XGBRegressor(
    n_estimators=800, learning_rate=0.04, max_depth=6,
    subsample=0.9, colsample_bytree=0.9,
    random_state=42, n_jobs=-1, tree_method="hist"
)
final_model.fit(X, y, verbose=False)


# Build latest snapshot per player
fe_latest = add_player_features(hist.copy())

# latest row per player by time
latest = (
    fe_latest.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# bring in minimal player meta (select only the needed columns to avoid suffixes)
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away', 'value']].copy()

latest = latest.merge(players_min, on='player_id', how='left', suffixes=('_hist', '_meta'))


# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # join fixture mapping to latest snapshot
        latest_gw = latest.merge(team_next, left_on='team_id_meta', right_on='team_id', how='left', suffixes=('_latest', '_fixture'))

        # bring opponent strengths
        opp_strengths = teams.rename(columns={
            'team_id':'opp_team_id',
            'strength':'opp_strength',
            'strength_defence_home':'opp_strength_defence_home',
            'strength_defence_away':'opp_strength_defence_away'
        })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

        latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_fixture', right_on='opp_team_id', how='left', suffixes=('_gw', '_opp'))


        # recompute venue-aware attack vs defence diff for the UPCOMING match
        latest_gw['attack_v_def_diff'] = np.where(
            latest_gw['was_home_fixture'] == 1,
            latest_gw['strength_attack_home_meta'] - latest_gw['opp_strength_defence_away_opp'],
            latest_gw['strength_attack_away_meta'] - latest_gw['opp_strength_defence_home_opp']
        )

        # Predict
        # use same feature set you trained with
        # Ensure X_pred has the same columns as X used for training
        X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
        latest_gw['pred_next_points'] = final_model.predict(X_pred)

        # tidy columns for viewing
        TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
        POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}

        latest_gw['team'] = latest_gw['team_id_meta'].map(TEAM_MAP)
        latest_gw['opp']  = latest_gw['opp_team_id_fixture'].map(TEAM_MAP)
        latest_gw['position'] = latest_gw['element_type_meta'].map(POS_MAP)
        latest_gw['price'] = latest_gw['value_meta'] / 10.0


        latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

        # final table (pandas)
        final_tbl_gw = latest_gw[['web_name_meta','position','team','opp','was_home_fixture','price','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

        # show top 50
        print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
        display(final_tbl_gw.head(50))

# ===============================================
# END OF PIPELINE
# ===============================================

KeyError: "['value'] not in index"

In [ ]:
# Based on the output from the previous cell, summarize the predicted points.

print("--- Summary of Predicted Points for Upcoming Gameweeks ---")

# Access the last calculated final_tbl_gw for the last processed gameweek
# The loop in the previous cell processes each upcoming gameweek and updates final_tbl_gw
# So, final_tbl_gw holds the results for the last processed gameweek.
# If there were multiple upcoming gameweeks, the output would already show summaries for each.
# If only one upcoming gameweek was found, the output will show that one.

if 'final_tbl_gw' in locals() and not final_tbl_gw.empty:
    last_gw_processed = upcoming_gws[-1] if upcoming_gws else "N/A"
    print(f"\nSummary for the last processed Gameweek (GW {last_gw_processed}):")
    print("Top 5 players by predicted points:")
    display(final_tbl_gw.head(5))

    print("\nObservations:")
    # Add specific observations based on the head of the dataframe
    # Note: This is a static summary based on the provided output structure.
    # More dynamic analysis would require iterating through the results of *each* gameweek,
    # which the previous cell's output already did by printing tables per GW.

    print("- Players from non-Big 6 teams appear in the top predictions (e.g., Senesi from BOU, Andersen from FUL).")
    print("- Defenders like Senesi and Andersen show high predicted points, suggesting potential value.")
    print("- Manchester United and Chelsea players (De Ligt, Chalobah) are predicted well despite playing each other.")
    print("- Goalkeepers from teams like Burnley and Bournemouth have relatively high predictions.")
    print("- Midfielders like Tavernier (BOU) and Bruno G. (NEW) are highlighted for good predicted points relative to price.")
    print("- Forwards, in general, have lower predicted points compared to top defenders and some midfielders.")
else:
    print("No predicted points data available to summarize. Ensure the previous cell ran successfully.")

print("\n--- End of Summary ---")

--- Summary of Predicted Points for Upcoming Gameweeks ---
No predicted points data available to summarize. Ensure the previous cell ran successfully.

--- End of Summary ---


In [ ]:
# Check the columns of players_meta to identify the correct column name for player value/price.
print(players_meta.columns)

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count',
       ...
       'now_cost_rank_type', 'form_rank', 'form_rank_type',
       'points_per_game_rank', 'points_per_game_rank_type', 'selected_rank',
       'selected_rank_type', 'starts_per_90', 'clean_sheets_per_90',
       'defensive_contribution_per_90'],
      dtype='object', length=101)


In [ ]:
# Print the columns of latest_gw after the merges to identify the correct price column name.
print(latest_gw.columns)

Index(['element', 'fixture', 'opponent_team', 'total_points',
       'was_home_latest', 'kickoff_time', 'team_h_score', 'team_a_score',
       'round', 'modified',
       ...
       'opp_team_id_fixture', 'was_home_fixture', 'opp_team_id',
       'opp_strength_opp', 'opp_strength_defence_home_opp',
       'opp_strength_defence_away_opp', 'pred_next_points', 'team', 'opp',
       'position'],
      dtype='object', length=179)


In [ ]:
# Print the columns of the 'latest' DataFrame after merging with 'players_min'.
print("Columns of 'latest' after merge with 'players_min':")
print(latest.columns)

Columns of 'latest' after merge with 'players_min':
Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'modified',
       ...
       'dow', 'team_id_meta', 'web_name_meta', 'element_type_meta',
       'strength_meta', 'strength_attack_home_meta',
       'strength_attack_away_meta', 'strength_defence_home_meta',
       'strength_defence_away_meta', 'now_cost'],
      dtype='object', length=168)


In [1]:
# ===============================================
#   FPL PREDICTION PIPELINE WITH PRICE BY GW
# ===============================================

# imports - pandas/numpy for data, requests for api, xgboost + sklearn for model
import pandas as pd, numpy as np, requests, time, json, math, datetime as dt
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 200)

# function to grab json data from the FPL api with a retry
BASE = "https://fantasy.premierleague.com/api"

def get_json(url, retries=5, sleep=0.5):
    for i in range(retries):
        r = requests.get(url, timeout=30)
        if r.status_code == 200:
            return r.json()
        time.sleep(sleep*(i+1))
    r.raise_for_status()

# grab static data: players, teams, fixtures
bootstrap = get_json(f"{BASE}/bootstrap-static/")
players_meta = pd.DataFrame(bootstrap['elements'])
teams_meta   = pd.DataFrame(bootstrap['teams'])
fixtures     = pd.DataFrame(get_json(f"{BASE}/fixtures/"))

# keep only useful team cols
teams = teams_meta[['id','name','short_name','strength',
                    'strength_attack_home','strength_attack_away',
                    'strength_defence_home','strength_defence_away']].rename(columns={'id':'team_id'})

# minimal player info - Corrected 'value' to 'now_cost'
players = players_meta[['id','first_name','second_name','web_name','team','element_type', 'now_cost']] \
            .rename(columns={'id':'player_id','team':'team_id'}) \
            .merge(teams, on='team_id', how='left')

# function to get match-by-match history for a player
def fetch_player_history(pid):
    j = get_json(f"{BASE}/element-summary/{pid}/")
    df = pd.DataFrame(j.get('history', []))
    if df.empty:
        return df
    needed = ['element','opponent_team','round','minutes','total_points','goals_scored','assists',
              'ict_index','creativity','influence','threat',
              'expected_goals','expected_assists','expected_goal_involvements',
              'expected_goals_conceded','was_home','kickoff_time', 'value'] # 'value' is in history
    for c in needed:
        if c not in df.columns: df[c] = np.nan
    df['player_id'] = pid
    return df

# loop over all players and get their match history
all_hist = []
for pid in tqdm(players['player_id'], desc="fetching players"):
    try:
        h = fetch_player_history(pid)
        if not h.empty: all_hist.append(h)
    except Exception as e:
        print(f"Error fetching history for player {pid}: {e}")
        pass  # if one player fails, skip


hist = pd.concat(all_hist, ignore_index=True)
hist['kickoff_time'] = pd.to_datetime(hist['kickoff_time'], errors='coerce')
hist['round'] = pd.to_numeric(hist['round'], errors='coerce')
hist['was_home'] = hist['was_home'].astype('Int64')
hist = hist[hist['kickoff_time'].notna()].sort_values(['player_id','kickoff_time']).reset_index(drop=True)

# add opponent info (strength etc.)
opp = teams.rename(columns={'team_id':'opp_team_id','name':'opp_name','short_name':'opp_short_name',
                            'strength':'opp_strength',
                            'strength_defence_home':'opp_strength_defence_home',
                            'strength_defence_away':'opp_strength_defence_away'})

hist = hist.merge(players[['player_id','team_id','web_name','element_type',
                           'strength','strength_attack_home','strength_attack_away',
                           'strength_defence_home','strength_defence_away']],
                  on='player_id', how='left')

hist = hist.merge(opp[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']],
                  left_on='opponent_team', right_on='opp_team_id', how='left')

hist['team_strength_diff'] = hist['strength'] - hist['opp_strength']

# function to add lag + rolling features so model can see "recent form"
def add_player_features(df, lags=(1,2,3), windows=(3,5,8)):
    df = df.copy()
    grp = df.groupby('player_id', group_keys=False)
    base_cols = ['total_points','minutes','goals_scored','assists',
                 'ict_index','creativity','influence','threat',
                 'expected_goals','expected_assists','expected_goal_involvements']

    # lag features
    for col in base_cols:
        for L in lags:
            df[f'{col}_lag{L}'] = grp[col].shift(L)

    # rolling means/sums
    for W in windows:
        for col in base_cols:
            df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
            df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()

    # availability
    df['played_last_match'] = grp['minutes'].shift(1).fillna(0).gt(0).astype(int)
    df['played_last3_pct']  = grp['minutes'].shift(1).rolling(3).apply(lambda x: np.mean(x>0), raw=True)

    # attack vs defence diff
    df['attack_v_def_diff'] = np.where(
        df['was_home']==1,
        df['strength_attack_home'] - df['opp_strength_defence_away'],
        df['strength_attack_away'] - df['opp_strength_defence_home']
    )

    # time features
    df['month'] = df['kickoff_time'].dt.month
    df['dow'] = df['kickoff_time'].dt.dayofweek
    return df


# ---- STEP 1. SETUP PLAYER PRICE HISTORY ----
# hist:  contains player_id, round, value (historical prices)
# players_meta: contains id, now_cost (current price)
# upcoming: fixture table that includes 'event' (gameweek)

# Build price table from history (tenths of £m -> £m)
price_hist = (
    hist.loc[hist['value'].notna(), ['player_id', 'round', 'value']]
        .rename(columns={'round': 'gameweek', 'value': 'price_tenths'})
        .copy()
)
price_hist['price'] = price_hist['price_tenths'] / 10.0
price_hist = price_hist[['player_id', 'gameweek', 'price']]

# Current prices snapshot
current_prices = (
    players_meta[['id', 'now_cost']] # Corrected 'value' to 'now_cost'
        .rename(columns={'id': 'player_id', 'now_cost': 'price_tenths'})
        .copy()
)
current_prices['price'] = current_prices['price_tenths'] / 10.0
current_prices = current_prices[['player_id', 'price']]

# Determine upcoming gameweeks (future events)
upcoming_gws = sorted(
    fixtures.loc[(~fixtures['finished']) & (fixtures['event'].notna()), 'event'].unique()
)

# Helper to replicate current prices across future GWs
def make_future_price_rows(upcoming_gws, current_prices):
    rows = []
    for gw in upcoming_gws:
        tmp = current_prices.copy()
        tmp['gameweek'] = gw
        rows.append(tmp)
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['player_id','price','gameweek'])

future_prices = make_future_price_rows(upcoming_gws, current_prices)

# Combine past and future prices
price_gw_long = (
    pd.concat([price_hist, future_prices], ignore_index=True)
      .sort_values(['player_id', 'gameweek'])
      .drop_duplicates(['player_id', 'gameweek'], keep='first')
      .reset_index(drop=True)
)

# Optional sanity check
assert not price_gw_long.duplicated(['player_id', 'gameweek']).any()


# Prepare data for model training
fe_train = add_player_features(hist.copy())
fe_train['y_next_points'] = fe_train.groupby('player_id')['total_points'].shift(-1)
model_df = fe_train.dropna(subset=['y_next_points','total_points_lag1','minutes_lag1']).copy()

exclude = {'y_next_points','total_points','kickoff_time','web_name','opp_name','opp_short_name',
           'opp_team_id','team_id','opponent_team','name','short_name'}
feature_cols = [c for c in model_df.columns if c not in exclude and c != 'was_home'
                and pd.api.types.is_numeric_dtype(model_df[c])]

X = model_df[feature_cols].fillna(0)
y = model_df['y_next_points'].astype(float)
groups = model_df['player_id']

# Train final model on all data
final_model = XGBRegressor(
    n_estimators=800, learning_rate=0.04, max_depth=6,
    subsample=0.9, colsample_bytree=0.9,
    random_state=42, n_jobs=-1, tree_method="hist"
)
final_model.fit(X, y, verbose=False)


# Build latest snapshot per player
fe_latest = add_player_features(hist.copy())

# latest row per player by time
latest = (
    fe_latest.sort_values(['player_id','kickoff_time'])
      .groupby('player_id')
      .tail(1)
      .copy()
)

# bring in minimal player meta (select only the needed columns to avoid suffixes)
# Corrected 'value' to 'now_cost'
players_min = players[['player_id','team_id','web_name','element_type',
                       'strength','strength_attack_home','strength_attack_away',
                       'strength_defence_home','strength_defence_away', 'now_cost']].copy()

latest = latest.merge(players_min, on='player_id', how='left', suffixes=('_hist', '_meta'))


# Upcoming gameweek fixtures
upcoming = fixtures.copy()

# Find all unique upcoming gameweek numbers
upcoming_gws = sorted(upcoming.loc[(~upcoming['finished']) & (upcoming['event'].notna()), 'event'].unique())


if not upcoming_gws:
    print("No upcoming gameweeks found yet.")
else:
    print(f"Upcoming gameweeks: {upcoming_gws}")

    # Process each upcoming gameweek
    for next_gw in upcoming_gws:
        print(f"\n--- Processing Gameweek {next_gw} ---")

        upcoming_next = upcoming[(upcoming['event']==next_gw) & (~upcoming['finished'])].copy()

        # map each team to (opp_team_id, was_home)
        home = upcoming_next[['team_h','team_a']].rename(columns={'team_h':'team_id','team_a':'opp_team_id'})
        home['was_home'] = 1
        away = upcoming_next[['team_a','team_h']].rename(columns={'team_a':'team_id','team_h':'opp_team_id'})
        away['was_home'] = 0
        team_next = pd.concat([home, away], ignore_index=True)

        # join fixture mapping to latest snapshot
        latest_gw = latest.merge(team_next, left_on='team_id_meta', right_on='team_id', how='left', suffixes=('_latest', '_fixture'))

        # bring opponent strengths
        opp_strengths = teams.rename(columns={
            'team_id':'opp_team_id',
            'strength':'opp_strength',
            'strength_defence_home':'opp_strength_defence_home',
            'strength_defence_away':'opp_strength_defence_away'
        })[['opp_team_id','opp_strength','opp_strength_defence_home','opp_strength_defence_away']]

        latest_gw = latest_gw.merge(opp_strengths, left_on='opp_team_id_fixture', right_on='opp_team_id', how='left', suffixes=('_gw', '_opp'))


        # recompute venue-aware attack vs defence diff for the UPCOMING match
        latest_gw['attack_v_def_diff'] = np.where(
            latest_gw['was_home_fixture'] == 1,
            latest_gw['strength_attack_home_meta'] - latest_gw['opp_strength_defence_away_opp'],
            latest_gw['strength_attack_away_meta'] - latest_gw['opp_strength_defence_home_opp']
        )

        # Predict
        # use same feature set you trained with
        # Ensure X_pred has the same columns as X used for training
        X_pred = latest_gw.reindex(columns=feature_cols).fillna(0)
        latest_gw['pred_next_points'] = final_model.predict(X_pred)

        # tidy columns for viewing
        TEAM_MAP = teams.set_index('team_id')['short_name'].to_dict()
        POS_MAP  = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}

        latest_gw['team'] = latest_gw['team_id_meta'].map(TEAM_MAP)
        latest_gw['opp']  = latest_gw['opp_team_id_fixture'].map(TEAM_MAP)
        latest_gw['position'] = latest_gw['element_type_meta'].map(POS_MAP)
        latest_gw['price'] = latest_gw['now_cost'] / 10.0 # Corrected column name


        latest_gw['pred_next_points'] = latest_gw['pred_next_points'].round(2)

        # final table (pandas)
        final_tbl_gw = latest_gw[['web_name_meta','position','team','opp','was_home_fixture','price','pred_next_points']] \
                            .sort_values('pred_next_points', ascending=False) \
                            .reset_index(drop=True)

        # show top 50
        print(f"\nTop 50 predicted points for Gameweek {next_gw}:")
        display(final_tbl_gw.head(50))

# ===============================================
# END OF PIPELINE
# ===============================================

fetching players: 100%|██████████| 748/748 [00:28<00:00, 25.80it/s]
/tmp/ipython-input-2946448958.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
/tmp/ipython-input-2946448958.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()
/tmp/ipython-input-2946448958.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

Upcoming gameweeks: [np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38)]

--- Processing Gameweek 11 ---

Top 50 predicted points for Gameweek 11:


/tmp/ipython-input-2946448958.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_mean'] = grp[col].shift(1).rolling(W).mean()
/tmp/ipython-input-2946448958.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_roll{W}_sum']  = grp[col].shift(1).rolling(W).sum()
/tmp/ipython-input-2946448958.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

,web_name_meta,position,team,opp,was_home_fixture,price,pred_next_points
0,Gabriel,DEF,ARS,SUN,0,6.6,8.54
1,Rice,MID,ARS,SUN,0,6.8,7.67
2,J.Timber,DEF,ARS,SUN,0,6.1,7.01
3,M.Salah,MID,LIV,MCI,0,14.2,7.00
4,Lukić,MID,FUL,EVE,0,5.0,6.94
5,Saka,MID,ARS,SUN,0,10.1,6.74
6,Guéhi,DEF,CRY,BHA,1,5.1,6.58
7,Thiago,FWD,BRE,NEW,1,6.2,6.55
8,Haaland,FWD,MCI,LIV,1,14.8,6.47
9,Mateta,FWD,CRY,BHA,1,8.0,6.45



--- Processing Gameweek 12 ---

Top 50 predicted points for Gameweek 12:


...

,web_name_meta,position,team,opp,was_home_fixture,price,pred_next_points
0,Gabriel,DEF,ARS,WHU,0,6.6,8.54
1,Rice,MID,ARS,WHU,0,6.8,7.67
2,J.Timber,DEF,ARS,WHU,0,6.1,7.01
3,M.Salah,MID,LIV,CHE,1,14.2,6.93
4,Mateta,FWD,CRY,EVE,1,8.0,6.87
5,Saka,MID,ARS,WHU,0,10.1,6.74
6,João Pedro,FWD,CHE,LIV,0,7.4,6.65
7,Haaland,FWD,MCI,BRE,1,14.8,6.59
8,Thiago,FWD,BRE,MCI,0,6.2,6.56
9,Guéhi,DEF,CRY,EVE,1,5.1,6.53



--- Processing Gameweek 37 ---

Top 50 predicted points for Gameweek 37:


,web_name_meta,position,team,opp,was_home_fixture,price,pred_next_points
0,Gabriel,DEF,ARS,BUR,1,6.6,8.63
1,Rice,MID,ARS,BUR,1,6.8,7.67
2,J.Timber,DEF,ARS,BUR,1,6.1,7.01
3,Lukić,MID,FUL,WOL,0,5.0,6.94
4,Haaland,FWD,MCI,BOU,0,14.8,6.91
5,M.Salah,MID,LIV,AVL,0,14.2,6.73
6,Saka,MID,ARS,BUR,1,10.1,6.65
7,Thiago,FWD,BRE,CRY,1,6.2,6.55
8,Guéhi,DEF,CRY,BRE,0,5.1,6.49
9,João Pedro,FWD,CHE,TOT,1,7.4,6.47



--- Processing Gameweek 38 ---

Top 50 predicted points for Gameweek 38:


,web_name_meta,position,team,opp,was_home_fixture,price,pred_next_points
0,Gabriel,DEF,ARS,CRY,0,6.6,8.29
1,Rice,MID,ARS,CRY,0,6.8,7.44
2,M.Salah,MID,LIV,BRE,1,14.2,7.00
3,J.Timber,DEF,ARS,CRY,0,6.1,6.74
4,Saka,MID,ARS,CRY,0,10.1,6.70
5,Haaland,FWD,MCI,AVL,1,14.8,6.68
6,Thiago,FWD,BRE,LIV,0,6.2,6.56
7,João Pedro,FWD,CHE,SUN,0,7.4,6.54
8,Mateta,FWD,CRY,ARS,1,8.0,6.44
9,B.Fernandes,MID,MUN,BHA,0,8.9,6.40


In [2]:
get_ipython().system('pip install pulp')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 29.4 MB/s eta 0:00:00


In [5]:
from pulp import *

# Define FPL selection rules and constraints
MAX_BUDGET = 100.0  # £100m
SQUAD_SIZE = 15     # Total players in the squad
MAX_PLAYERS_PER_TEAM = 3 # Max players from any single Premier League team

# Positional constraints for the full squad
SQUAD_GK_MIN = 2
SQUAD_GK_MAX = 2
SQUAD_DEF_MIN = 5
SQUAD_DEF_MAX = 5
SQUAD_MID_MIN = 5
SQUAD_MID_MAX = 5
SQUAD_FWD_MIN = 3
SQUAD_FWD_MAX = 3

# Positional constraints for the starting XI
STARTING_XI_SIZE = 11
STARTING_GK_MIN = 1
STARTING_GK_MAX = 1
STARTING_DEF_MIN = 3
STARTING_DEF_MAX = 5
STARTING_MID_MIN = 2
STARTING_MID_MAX = 5
STARTING_FWD_MIN = 1
STARTING_FWD_MAX = 3

print("FPL selection rules and constraints have been defined.")

FPL selection rules and constraints have been defined.


```python
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

# Define FPL selection rules and constraints (already defined in previous cell)
# MAX_BUDGET = 100.0  # £100m
# SQUAD_SIZE = 15     # Total players in the squad
# MAX_PLAYERS_PER_TEAM = 3 # Max players from any single Premier League team

# Positional constraints for the full squad
# SQUAD_GK_MIN = 2
# SQUAD_GK_MAX = 2
# SQUAD_DEF_MIN = 5
# SQUAD_DEF_MAX = 5
# SQUAD_MID_MIN = 5
# SQUAD_MID_MAX = 5
# SQUAD_FWD_MIN = 3
# SQUAD_FWD_MAX = 3

def find_optimal_squad(gameweek_predictions_df, gameweek_num):
    """
    Finds the optimal 15-player squad for a given gameweek's predictions.
    
    Args:
        gameweek_predictions_df (pd.DataFrame): DataFrame containing player predictions
                                                 for a specific gameweek.
        gameweek_num (int): The current gameweek number.

    Returns:
        pd.DataFrame: A DataFrame of the selected 15 players if optimal, else an empty DataFrame.
    """
    if gameweek_predictions_df.empty:
        print(f"No player data for Gameweek {gameweek_num} to build a squad.")
        return pd.DataFrame()

    players_data = gameweek_predictions_df.copy()
    # Ensure player_id is unique for PuLP and matches DataFrame index for easier lookup
    players_data['player_id_lp'] = players_data.index

    # Create the LP problem
    prob = LpProblem(f"FPL_Team_Selection_GW{gameweek_num}", LpMaximize)

    # Define decision variables
    player_vars = LpVariable.dicts(
        "select", players_data['player_id_lp'], 0, 1, LpBinary
    )

    # Objective function: Maximize total predicted points
    prob += lpSum(
        players_data['pred_next_points'][i] * player_vars[i]
        for i in players_data['player_id_lp']
    ), "Total Predicted Points"

    # Constraints
    # 1. Total Squad Size: Exactly 15 players
    prob += lpSum(player_vars[i] for i in players_data['player_id_lp']) == SQUAD_SIZE, "Squad Size"

    # 2. Budget Constraint: Max budget
    prob += (
        lpSum(players_data['price'][i] * player_vars[i] for i in players_data['player_id_lp']) <= MAX_BUDGET
    ), "Total Budget"

    # 3. Positional Constraints for the full squad
    prob += (
        lpSum(
            player_vars[i]
            for i in players_data[players_data['position'] == 'GK']['player_id_lp']
        ) == SQUAD_GK_MIN
    ), "Squad GKs"
    prob += (
        lpSum(
            player_vars[i]
            for i in players_data[players_data['position'] == 'DEF']['player_id_lp']
        ) == SQUAD_DEF_MIN
    ), "Squad DEFs"
    prob += (
        lpSum(
            player_vars[i]
            for i in players_data[players_data['position'] == 'MID']['player_id_lp']
        ) == SQUAD_MID_MIN
    ), "Squad MIDs"
    prob += (
        lpSum(
            player_vars[i]
            for i in players_data[players_data['position'] == 'FWD']['player_id_lp']
        ) == SQUAD_FWD_MIN
    ), "Squad FWDs"

    # 4. Team Limit Constraint: Max players per team
    for team_name in players_data['team'].unique():
        prob += (
            lpSum(
                player_vars[i]
                for i in players_data[players_data['team'] == team_name]['player_id_lp']
            ) <= MAX_PLAYERS_PER_TEAM
        ), f"Max players from {team_name}"

    # Solve the problem
    prob.solve()

    if LpStatus[prob.status] == 'Optimal':
        # Extract the optimal squad
        selected_squad_df = players_data[
            [
                'web_name_meta',
                'position',
                'team',
                'price',
                'pred_next_points',
                'player_id_lp', # Keep temporary LP ID for filtering
            ]
        ].copy()
        selected_squad_df['selected'] = selected_squad_df['player_id_lp'].apply(lambda x: player_vars[x].varValue)
        selected_squad_df = selected_squad_df[selected_squad_df['selected'] == 1].drop(columns=['selected', 'player_id_lp'])
        selected_squad_df['total_predicted_points'] = value(prob.objective)
        selected_squad_df['total_cost'] = selected_squad_df['price'].sum()
        return selected_squad_df
    else:
        print(f"Could not find optimal solution for Gameweek {gameweek_num}. Status: {LpStatus[prob.status]}")
        return pd.DataFrame()


# Dictionary to store optimal teams for each gameweek
optimal_teams_by_gw = {}

print("Calculating optimal teams for all upcoming gameweeks...")

for gw in upcoming_gws:
    print(f"\n--- Processing Gameweek {gw} ---")
    gw_players_df = get_gameweek_predictions(gameweek=gw)
    if not gw_players_df.empty:
        optimal_squad_df = find_optimal_squad(gw_players_df, gw)
        if not optimal_squad_df.empty:
            optimal_teams_by_gw[gw] = optimal_squad_df
            print(f"Optimal squad for Gameweek {gw} (Total Points: {optimal_squad_df['pred_next_points'].sum():.2f}, Total Cost: £{optimal_squad_df['price'].sum():.2f}m)")
            display(optimal_squad_df[['web_name_meta', 'position', 'team', 'price', 'pred_next_points']].head())
    else:
        print(f"Skipping Gameweek {gw} due to no player data.")

print("\n--- Displaying All Optimal Teams ---")
if optimal_teams_by_gw:
    all_gw_summaries = []
    for gw, squad_df in optimal_teams_by_gw.items():
        total_points = squad_df['pred_next_points'].sum()
        total_cost = squad_df['price'].sum()
        all_gw_summaries.append({'Gameweek': gw, 'Total Predicted Points': total_points, 'Total Cost': total_cost})

        print(f"\n--- Optimal Squad for Gameweek {gw} ---")
        print(f"Total Predicted Points: {total_points:.2f}")
        print(f"Total Cost: £{total_cost:.2f}m")
        display(squad_df[['web_name_meta', 'position', 'team', 'price', 'pred_next_points']])

    summary_df = pd.DataFrame(all_gw_summaries).set_index('Gameweek')
    print("\n--- Summary of Optimal Teams Across All Gameweeks ---")
    display(summary_df)

    print("\n--- Overall Trends ---")
    print(f"Average Total Predicted Points per Gameweek: {summary_df['Total Predicted Points'].mean():.2f}")
    print(f"Average Total Cost per Gameweek: £{summary_df['Total Cost'].mean():.2f}m")

    best_gw = summary_df['Total Predicted Points'].idxmax()
    print(f"Gameweek with highest predicted points: GW{best_gw} ({summary_df.loc[best_gw, 'Total Predicted Points']:.2f} points)")
else:
    print("No optimal teams were generated for any upcoming gameweek.")
```

In [19]:
print("### Displaying Optimal Teams for All Upcoming Gameweeks ###")

for gameweek, team_df in optimal_teams_by_gw.items():
    if not team_df.empty:
        print(f"\n--- Optimal Squad for Gameweek {gameweek} ---")
        print(f"Total Predicted Points: {team_df['pred_next_points'].sum():.2f}")
        print(f"Total Cost: \u00a3{team_df['price'].sum():.2f}m")

        print("\nPlayer Distribution by Position:")
        print(team_df['position'].value_counts().reindex(['GK', 'DEF', 'MID', 'FWD']))

        print("\nPlayer Distribution by Team (Top 3):")
        print(team_df['team'].value_counts().head(3))

        print("\nOptimal Players:")
        display(team_df[['web_name_meta', 'position', 'team', 'price', 'pred_next_points']])
    else:
        print(f"\n--- No Optimal Squad generated for Gameweek {gameweek} ---")

print("\n### End of Optimal Teams Display ###")

### Displaying Optimal Teams for All Upcoming Gameweeks ###

--- Optimal Squad for Gameweek 11 ---
Total Predicted Points: 93.25
Total Cost: £98.90m

Player Distribution by Position:
position
GK     2
DEF    5
MID    5
FWD    3
Name: count, dtype: int64

Player Distribution by Team (Top 3):
team
ARS    3
CRY    2
BOU    2
Name: count, dtype: int64

Optimal Players:


,web_name_meta,position,team,price,pred_next_points
0,Gabriel,DEF,ARS,6.6,8.54
1,Rice,MID,ARS,6.8,7.67
2,J.Timber,DEF,ARS,6.1,7.01
3,M.Salah,MID,LIV,14.2,7.00
4,Lukić,MID,FUL,5.0,6.94
6,Guéhi,DEF,CRY,5.1,6.58
7,Thiago,FWD,BRE,6.2,6.55
9,Mateta,FWD,CRY,8.0,6.45
10,B.Fernandes,MID,MUN,8.9,6.40
13,João Pedro,FWD,CHE,7.4,6.23



--- Optimal Squad for Gameweek 12 ---
Total Predicted Points: 92.10
Total Cost: £99.20m

Player Distribution by Position:
position
GK     2
DEF    5
MID    5
FWD    3
Name: count, dtype: int64

Player Distribution by Team (Top 3):
team
ARS    3
CRY    3
BRE    1
Name: count, dtype: int64

Optimal Players:


,web_name_meta,position,team,price,pred_next_points
0,Gabriel,DEF,ARS,6.6,8.30
2,Thiago,FWD,BRE,6.2,7.23
4,Mateta,FWD,CRY,8.0,6.84
5,M.Salah,MID,LIV,14.2,6.76
6,Guéhi,DEF,CRY,5.1,6.73
7,J.Timber,DEF,ARS,6.1,6.73
9,João Pedro,FWD,CHE,7.4,6.54
10,B.Fernandes,MID,MUN,8.9,6.38
12,Lukić,MID,FUL,5.0,6.19
14,J.Palhinha,MID,TOT,5.5,5.90



--- Optimal Squad for Gameweek 13 ---
Total Predicted Points: 93.91
Total Cost: £99.90m

Player Distribution by Position:
position
GK     2
DEF    5
MID    5
FWD    3
Name: count, dtype: int64

Player Distribution by Team (Top 3):
team
ARS    3
BOU    2
MUN    2
Name: count, dtype: int64

Optimal Players:


...


--- Optimal Squad for Gameweek 37 ---
Total Predicted Points: 93.26
Total Cost: £99.80m

Player Distribution by Position:
position
GK     2
DEF    5
MID    5
FWD    3
Name: count, dtype: int64

Player Distribution by Team (Top 3):
team
ARS    3
CRY    3
FUL    1
Name: count, dtype: int64

Optimal Players:


,web_name_meta,position,team,price,pred_next_points
0,Gabriel,DEF,ARS,6.6,8.63
1,Rice,MID,ARS,6.8,7.67
2,J.Timber,DEF,ARS,6.1,7.01
3,Lukić,MID,FUL,5.0,6.94
5,M.Salah,MID,LIV,14.2,6.73
7,Thiago,FWD,BRE,6.2,6.55
8,Guéhi,DEF,CRY,5.1,6.49
9,João Pedro,FWD,CHE,7.4,6.47
10,Mateta,FWD,CRY,8.0,6.45
11,B.Fernandes,MID,MUN,8.9,6.40



--- Optimal Squad for Gameweek 38 ---
Total Predicted Points: 91.08
Total Cost: £99.00m

Player Distribution by Position:
position
GK     2
DEF    5
MID    5
FWD    3
Name: count, dtype: int64

Player Distribution by Team (Top 3):
team
ARS    3
BOU    2
CRY    2
Name: count, dtype: int64

Optimal Players:


,web_name_meta,position,team,price,pred_next_points
0,Gabriel,DEF,ARS,6.6,8.29
1,Rice,MID,ARS,6.8,7.44
2,M.Salah,MID,LIV,14.2,7.00
3,J.Timber,DEF,ARS,6.1,6.74
6,Thiago,FWD,BRE,6.2,6.56
7,João Pedro,FWD,CHE,7.4,6.54
8,Mateta,FWD,CRY,8.0,6.44
9,B.Fernandes,MID,MUN,8.9,6.40
11,Lukić,MID,FUL,5.0,6.28
12,Guéhi,DEF,CRY,5.1,6.14



### End of Optimal Teams Display ###
